In [1]:
from lib.main_func_p1 import path, timer
import pandas as pd
from importlib import reload
from collections import OrderedDict
import lib.main_func_p4
from lib.main_func_p4 import modelXGBoost_fit_scores


#XGBoost library
import xgboost as xgb


In [2]:
# proteina (uniprot_ID)
uniprot_id = 'P22303'
excel_name = 'P22303_20220518151513_top_scores_XGBClassifier'
path_file = path(uniprot_id)
# Parametros
seed = 142857
fp_name = 'morgan2_c'

In [3]:
# Load top_scores
top_scores = pd.read_excel(f'./top_scores/{excel_name}.xlsx')

# Best calibration scores
top_socres_len = 15
top_scores_top = top_scores.iloc[:top_socres_len]
top_scores_top_index_list = top_scores_top.index
top_scores

,model,params_dict,AUC_train,AUC_valid,accuracy_train,accuracy_valid,recall_train,recall_valid,specificity_train,specificity_valid,precision_train,precision_valid,f1_score_train,f1_score_valid,conf_matrix_train,conf_matrix_valid,calibration_score
0,modelID_702,"OrderedDict([('alpha', 0.0), ('gamma', 0.0), (...",0.994,0.965,0.974,0.906,0.972,0.877,0.976,0.937,0.978,0.938,0.974,0.906,"[[1406, 34], [43, 1520]]","[[239, 16], [34, 242]]",0.021748
1,modelID_574,"OrderedDict([('alpha', 0.5), ('gamma', 0.0), (...",0.995,0.964,0.976,0.906,0.974,0.884,0.978,0.930,0.979,0.931,0.976,0.906,"[[1408, 32], [41, 1522]]","[[237, 18], [32, 244]]",0.022024
2,modelID_400,"OrderedDict([('alpha', 0.0), ('gamma', 0.2), (...",0.994,0.960,0.976,0.893,0.977,0.866,0.975,0.922,0.977,0.923,0.976,0.893,"[[1404, 36], [36, 1527]]","[[235, 20], [37, 239]]",0.022161
3,modelID_441,"OrderedDict([('alpha', 0.2), ('gamma', 0.1), (...",0.994,0.960,0.977,0.904,0.976,0.891,0.978,0.918,0.979,0.921,0.977,0.904,"[[1408, 32], [38, 1525]]","[[234, 21], [30, 246]]",0.022722
4,modelID_471,"OrderedDict([('alpha', 0.2), ('gamma', 0.2), (...",0.995,0.958,0.974,0.895,0.976,0.888,0.972,0.903,0.973,0.907,0.974,0.895,"[[1398, 42], [37, 1526]]","[[230, 25], [31, 245]]",0.022794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,modelID_4,"OrderedDict([('alpha', 2.0), ('gamma', 12.0), ...",0.913,0.898,0.839,0.793,0.832,0.736,0.847,0.855,0.856,0.846,0.840,0.792,"[[1221, 219], [263, 1300]]","[[218, 37], [73, 203]]",0.134077
764,modelID_3,"OrderedDict([('alpha', 2.0), ('gamma', 12.0), ...",0.883,0.878,0.802,0.797,0.788,0.714,0.817,0.887,0.824,0.872,0.802,0.796,"[[1177, 263], [332, 1231]]","[[226, 29], [79, 197]]",0.164325
765,modelID_1,"OrderedDict([('alpha', 2.0), ('gamma', 24.0), ...",0.863,0.855,0.784,0.770,0.743,0.681,0.829,0.866,0.825,0.847,0.784,0.769,"[[1194, 246], [402, 1161]]","[[221, 34], [88, 188]]",0.179674
766,modelID_2,"OrderedDict([('alpha', 0.2), ('gamma', 24.0), ...",0.823,0.812,0.754,0.727,0.706,0.630,0.806,0.832,0.797,0.802,0.753,0.725,"[[1159, 281], [459, 1104]]","[[212, 43], [102, 174]]",0.180192


In [ ]:
reload(lib.main_func_p4)
from lib.main_func_p4 import modelXGBoost_fit_scores
# Load train and validation datasets
df_train = pd.read_pickle(f'{path_file}_dataset_train')
df_valid = pd.read_pickle(f'{path_file}_dataset_valid')

new_row_list = list()
results_ROF_list_train = list()
results_ROF_list_valid = list()
df_list = list()
plots_name_list = list()

for i, params_dict in enumerate(top_scores.params_dict.iloc[0:top_socres_len]):
    eval_metric = ['error', 'auc']
    tick = timer()
    params_dict_Orderer = params_dict
    params_dict = dict(eval(params_dict))
    default_params_xgb = {'booster': 'gbtree', 'tree_method': 'gpu_hist',
                          'objective':'binary:logistic', 'grow_policy': 'depthwise',
                          'eval_metric': eval_metric, 'early_stopping_rounds':10}
    params_dict.update(default_params_xgb)


    xgb_clf = xgb.XGBClassifier(**params_dict)

    # Train model and evaluating scores (train / validation)
    xgb_clf, scores_train, scores_valid = modelXGBoost_fit_scores(xgb_clf, fp_name, df_train, df_valid,
                                                                  resample_factor=0,
                                                                  resample_mode='under_sampling')

    if i == 0:
        xgb_clf.save_model(f'./models/{uniprot_id}_model.ubj')
        print(f'Best saved ({top_scores.model.iloc[i]})')

    # ROF results
    results_ROF_list_train.append(scores_train[7])
    results_ROF_list_valid.append(scores_valid[7])

    # save pred and pred_prob of train set
    df = df_train[['activity', 'prediction', 'prediction_prob']].copy()
    df_list.append(df)

    plots_name_list.append(top_scores.model.iloc[i])
    print(f'{i}/{top_socres_len}. {top_scores.model.iloc[i]}. AUC_socre=(train={scores_train[0]},'
          f' valid={scores_valid[0]}). Time elapsed: {timer(tick)}')

Best saved (modelID_702)
0/15. modelID_702. AUC_socre=(train=0.996, valid=0.954). Time elapsed: 8.2 seconds.
1/15. modelID_574. AUC_socre=(train=0.993, valid=0.957). Time elapsed: 7.5 seconds.
2/15. modelID_400. AUC_socre=(train=0.996, valid=0.96). Time elapsed: 3.4 seconds.
3/15. modelID_441. AUC_socre=(train=0.993, valid=0.959). Time elapsed: 3.0 seconds.
4/15. modelID_471. AUC_socre=(train=0.996, valid=0.961). Time elapsed: 2.9 seconds.
5/15. modelID_606. AUC_socre=(train=0.994, valid=0.956). Time elapsed: 2.9 seconds.


In [ ]:
# ROC curve - best scores
reload(lib.main_func_p4)
from lib.main_func_p4 import plot_ROC_curve
results_ROF_train_list_top = list()
results_ROF_valid_list_top = list()
ROC_plots_name_top_name = list()
for i in top_scores_top_index_list:
    results_ROF_train_list_top.append(results_ROF_list_train[i])
    results_ROF_valid_list_top.append(results_ROF_list_valid[i])
    ROC_plots_name_top_name.append(plots_name_list[i])

model_name = f'XGBoost_Clf Train set (TOP)'
plot_ROC_curve(results_ROF_train_list_top, ROC_plots_name_top_name, model_name,
               path_file=path_file, name_mod="train_Top_score", save_fig=True)

In [ ]:
model_name = f'XGBoost_Clf validation set (TOP)'
plot_ROC_curve(results_ROF_valid_list_top, ROC_plots_name_top_name, model_name,
               path_file=path_file, name_mod="validation_Top_score", save_fig=True)

In [ ]:
# Calibration curve - best scores (10)
reload(lib.main_func_p4)
from lib.main_func_p4 import plot_calibration_curve
df_list_top = list()
df_list_top_name = list()

# Only graph top 5
for i in top_scores_top_index_list:
    df_list_top.append(df_list[i])
    df_list_top_name.append((plots_name_list[i]))
plot_calibration_curve(df_list_top[:5], df_list_top_name[:5], model_name,
                       path_file=path_file, name_mod="validation_Top_score", save_fig=True)